# Advanced and Specialized Visualizations — Solution Notebook

This notebook contains **complete solutions** for all code cells and exercises in
`notebooks/03_advanced_visualizations.ipynb`.

- **Part 1:** VAR Visualizations — IRFs, FEVD, Stability, Granger Causality Network
- **Part 2:** Quantile Regression Visualizations — Coefficient Paths β(τ), Fan Charts, Interactive Dashboard
- **Part 3:** Spatial Visualizations — Moran Scatterplot, Spatial Weights Network, Choropleth
- **Part 4:** Model Comparison — Coefficient Plots, Forest Plots, IC Comparison

**API notes:**
- `var_results.irf(periods=, method=, ci_method=)` — not `horizon=`
- `var_results.fevd(periods=)` — not `horizon=`
- `create_comparison_charts(charts=['coefficients', 'forest_plot', 'fit_comparison', 'ic_comparison'])` — not `coefficient_comparison`
- `PooledQuantile.params` shape is `(n_params, n_quantiles)` for multiple quantiles
- `var_results.plot_stability()` — method on result, not standalone function
- `var_results.plot_causality_network(threshold=, layout=, backend=)` — method on result

In [ ]:
import sys
import os
sys.path.insert(0, '../../../')
sys.path.insert(0, '../')

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'

np.random.seed(42)

from utils.data_generators import (
    make_var_data,
    make_quantile_wage_panel,
    make_spatial_data,
    generate_panel_data,
)

os.makedirs('../outputs/charts/png', exist_ok=True)
os.makedirs('../outputs/charts/svg', exist_ok=True)
os.makedirs('../outputs/reports/html', exist_ok=True)

print('Setup complete.')

---
## Part 1: VAR Visualizations

In [ ]:
from panelbox.var import PanelVAR
from panelbox.var.data import PanelVARData

# 1. Generate quarterly macro data
df_var = make_var_data(n_obs=200, n_vars=3, seed=42)
print(df_var.head())
print(f'\nShape: {df_var.shape}')
print(f'Variables: {df_var.columns.tolist()}')
print(f'Date range: {df_var.index[0]} to {df_var.index[-1]}')

In [ ]:
# 2. Build PanelVARData — treat the time series as a single-entity panel
# PanelVARData requires entity_col and time_col; DatetimeIndex is not directly supported
df_var_panel = df_var.copy().reset_index()
df_var_panel.columns = ['period'] + list(df_var.columns)
df_var_panel['entity'] = 1

var_data = PanelVARData(
    data=df_var_panel,
    endog_vars=['gdp_growth', 'inflation', 'interest_rate'],
    entity_col='entity',
    time_col='period',
    lags=2,
    trend='constant',
)

var_model   = PanelVAR(data=var_data)
var_results = var_model.fit(cov_type='nonrobust')

print('VAR(2) estimated.')
print(var_results.summary())

### 1.1 Impulse Response Functions

In [ ]:
from panelbox.visualization.var_plots import plot_irf

# Compute IRFs without bootstrap (fast)
irf_result = var_results.irf(
    periods=20,
    method='cholesky',
    ci_method=None,
)

fig_irf = plot_irf(
    irf_result=irf_result,
    ci=False,
    backend='matplotlib',
    theme='academic',
    show=False,
)

if fig_irf is not None:
    fig_irf.tight_layout()
    fig_irf.savefig('../outputs/charts/png/03_var_irf.png', dpi=150, bbox_inches='tight')
    print('IRF chart saved.')
    plt.show()
else:
    print('plot_irf returned None (show was True or backend returned None).')

### Exercise 1 — Solution: IRFs with bootstrap CIs, response of GDP only

In [ ]:
# Exercise 1 solution
# Re-compute with bootstrap CI (n_bootstrap=200 for speed) and plot only gdp_growth responses

irf_boot = var_results.irf(
    periods=20,
    method='cholesky',
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

# Plot only the responses of gdp_growth to all shocks (1-row grid)
fig_irf_gdp = plot_irf(
    irf_result=irf_boot,
    response='gdp_growth',   # filter: only show responses of this variable
    ci=True,
    backend='matplotlib',
    theme='academic',
    show=False,
)

if fig_irf_gdp is not None:
    fig_irf_gdp.suptitle('IRF: Responses of GDP Growth (with 95% Bootstrap CI)', y=1.02)
    fig_irf_gdp.tight_layout()
    fig_irf_gdp.savefig('../outputs/charts/png/03_var_irf_gdp_boot.png',
                        dpi=150, bbox_inches='tight')
    print('Bootstrap IRF saved.')
    plt.show()
else:
    print('plot_irf returned None.')

### 1.2 Forecast Error Variance Decomposition

In [ ]:
from panelbox.visualization.var_plots import plot_fevd

fevd_result = var_results.fevd(periods=20, method='cholesky')

fig_fevd = plot_fevd(
    fevd_result=fevd_result,
    kind='area',
    backend='matplotlib',
    theme='academic',
    show=False,
)

if fig_fevd is not None:
    fig_fevd.tight_layout()
    fig_fevd.savefig('../outputs/charts/png/03_var_fevd.png', dpi=150, bbox_inches='tight')
    print('FEVD chart saved.')
    plt.show()
else:
    print('plot_fevd returned None.')

### 1.3 Stability Diagram

In [ ]:
eigenvalues = var_results.eigenvalues
moduli      = np.abs(eigenvalues)
print(f'Eigenvalue moduli: {moduli.round(4)}')
print(f'Maximum modulus  : {moduli.max():.4f}')
print(f'System stable    : {moduli.max() < 1}')

# Method on result object — no need to extract eigenvalues separately
fig_stab = var_results.plot_stability(backend='matplotlib', show=False)

if fig_stab is not None:
    fig_stab.tight_layout()
    fig_stab.savefig('../outputs/charts/png/03_var_stability.png', dpi=150, bbox_inches='tight')
    print('Stability chart saved.')
    plt.show()
else:
    # Fallback: draw manually using the eigenvalues property
    fig, ax = plt.subplots(figsize=(6, 6))
    theta = np.linspace(0, 2 * np.pi, 300)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', lw=1.5, label='Unit circle')
    ax.axhline(0, color='gray', lw=0.5); ax.axvline(0, color='gray', lw=0.5)
    colors = ['green' if m < 1 else 'red' for m in moduli]
    ax.scatter(eigenvalues.real, eigenvalues.imag, c=colors, s=80, zorder=5,
               label='Eigenvalues')
    ax.set_xlim(-1.3, 1.3); ax.set_ylim(-1.3, 1.3); ax.set_aspect('equal')
    ax.set_title('VAR Stability — Roots of Companion Matrix')
    ax.set_xlabel('Real part'); ax.set_ylabel('Imaginary part')
    ax.legend()
    fig.tight_layout()
    fig.savefig('../outputs/charts/png/03_var_stability.png', dpi=150, bbox_inches='tight')
    plt.show()

### 1.4 Granger Causality Network

In [ ]:
# All pairwise Granger causality p-values
gc_matrix = var_results.granger_causality_matrix(significance_level=0.05)
print('Granger Causality p-value matrix:')
print('Row = cause, Column = effect')
print(gc_matrix.round(4))

In [ ]:
# Interactive directed graph — method on PanelVARResult
fig_gc_net = var_results.plot_causality_network(
    threshold=0.10,
    layout='circular',
    backend='plotly',
    show=True,
)

In [ ]:
# Individual pairwise tests
variables = ['gdp_growth', 'inflation', 'interest_rate']
print('Pairwise Granger causality tests (VAR(2)):')
print('=' * 60)
for cause in variables:
    for effect in variables:
        if cause != effect:
            gc = var_results.granger_causality(cause=cause, effect=effect)
            sig = ('***' if gc.p_value < 0.01 else
                   '**'  if gc.p_value < 0.05 else
                   '*'   if gc.p_value < 0.10 else '')
            print(f'  {cause:>14} → {effect:<14}: '
                  f'F={gc.f_stat:.3f}  p={gc.p_value:.4f} {sig}')

---
## Part 2: Quantile Regression Visualizations

In [ ]:
from panelbox.models.quantile import PooledQuantile

df_wage       = make_quantile_wage_panel(n_entities=200, n_periods=5, seed=42)
df_wage_reset = df_wage.reset_index()

print(df_wage_reset.head())
print(f'\nShape     : {df_wage_reset.shape}')
print(f'Columns   : {df_wage_reset.columns.tolist()}')
print(f'Wage range: [{df_wage_reset["wage"].min():.2f}, {df_wage_reset["wage"].max():.2f}]')
print(f'Educ range: [{df_wage_reset["educ"].min()}, {df_wage_reset["educ"].max()}] years')

In [ ]:
# Fit PooledQuantile at 5 quantiles simultaneously
# PooledQuantile does NOT add an intercept — must include one in exog
quantiles    = np.array([0.10, 0.25, 0.50, 0.75, 0.90])
X_wages      = df_wage_reset[['educ', 'exper']].values
y_wages      = df_wage_reset['wage'].values
X_with_const = np.column_stack([np.ones(len(y_wages)), X_wages])
param_names  = ['Intercept', 'Education', 'Experience']

qr_model  = PooledQuantile(
    endog=y_wages,
    exog=X_with_const,
    entity_id=df_wage_reset['entity'].values,
    time_id=df_wage_reset['time'].values,
    quantiles=quantiles,
)
qr_result = qr_model.fit(se_type='cluster')

# params: shape (n_params=3, n_quantiles=5)
print(f'params shape: {qr_result.params.shape}  (n_params × n_quantiles)')
params_df = pd.DataFrame(
    qr_result.params,
    index=param_names,
    columns=[f'τ={q:.2f}' for q in quantiles],
)
print('\nEstimated coefficients by quantile:')
print(params_df.round(4))

### 2.1 Coefficient Path β(τ)

In [ ]:
from panelbox.visualization.quantile.advanced_plots import QuantileVisualizer

qv = QuantileVisualizer(style='academic', dpi=150)

try:
    fig_coef_path = qv.coefficient_path(
        result=qr_result,
        var_names=param_names,
        uniform_bands=True,
    )
    fig_coef_path.suptitle('Coefficient Paths Across Quantiles β(τ)', y=1.02)
    fig_coef_path.tight_layout()
    fig_coef_path.savefig('../outputs/charts/png/03_qr_coef_path.png',
                          dpi=150, bbox_inches='tight')
    print('Coefficient path saved.')
    plt.show()
except Exception as e:
    print(f'QuantileVisualizer error: {e} — manual fallback.')
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    for idx, (ax, name) in enumerate(zip(axes, param_names)):
        coefs = qr_result.params[idx, :]    # shape: (n_quantiles,)
        ax.plot(quantiles, coefs, 'o-', color='steelblue', lw=2, ms=6)
        ax.axhline(np.mean(coefs), color='red', ls='--', lw=1, label='Mean')
        ax.set_title(name); ax.set_xlabel('Quantile τ'); ax.set_ylabel('Coefficient')
        ax.legend(fontsize=9)
    fig.suptitle('Coefficient Paths Across Quantiles β(τ)')
    fig.tight_layout()
    fig.savefig('../outputs/charts/png/03_qr_coef_path.png', dpi=150, bbox_inches='tight')
    plt.show()

**Key insight:** The upward-sloping Education path confirms that returns to education are
larger for high-wage workers. OLS captures only the average (flat line).

### 2.2 Fan Chart

In [ ]:
educ_range = np.linspace(8, 20, 60)
avg_exper  = df_wage_reset['exper'].mean()

X_forecast = np.column_stack([
    np.ones(len(educ_range)),
    educ_range,
    np.full(len(educ_range), avg_exper),
])

try:
    fig_fan = qv.fan_chart(
        result=qr_result,
        X_forecast=X_forecast,
        time_index=educ_range,
        tau_list=list(quantiles),
        alpha_gradient=True,
    )
    fig_fan.axes[0].set_xlabel('Years of Education')
    fig_fan.axes[0].set_ylabel('Predicted Wage')
    fig_fan.axes[0].set_title('Wage Distribution by Education Level — Fan Chart')
    fig_fan.tight_layout()
    fig_fan.savefig('../outputs/charts/png/03_qr_fan_chart.png', dpi=150, bbox_inches='tight')
    print('Fan chart saved.')
    plt.show()
except Exception as e:
    print(f'fan_chart error: {e} — Plotly fallback.')
    colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4', '#9467bd']
    fig_fan_plotly = go.Figure()
    for i, tau in enumerate(quantiles):
        # Prediction: y_hat(x) = X_forecast @ params[:, i]
        y_pred = X_forecast @ qr_result.params[:, i]
        fig_fan_plotly.add_trace(go.Scatter(
            x=educ_range, y=y_pred,
            mode='lines', name=f'τ = {tau:.2f}',
            line=dict(color=colors[i], width=2.5),
        ))
    fig_fan_plotly.update_layout(
        title='Wage Distribution by Education Level — Fan Chart',
        xaxis_title='Years of Education', yaxis_title='Predicted Wage',
        template='plotly_white', font=dict(family='Arial', size=13),
    )
    fig_fan_plotly.show()

### 2.3 Interactive Coefficient Dashboard

In [ ]:
from panelbox.visualization.quantile.interactive import InteractivePlotter

try:
    ip       = InteractivePlotter(theme='plotly_white')
    dash_fig = ip.coefficient_dashboard(result=qr_result, var_names=param_names)
    dash_fig.show()
    dash_fig.write_html('../outputs/reports/html/03_quantile_dashboard.html',
                        include_plotlyjs=True)
    print('Interactive dashboard saved.')
except Exception as e:
    print(f'InteractivePlotter error: {e} — manual Plotly slider.')
    frames = [
        go.Frame(
            data=[go.Bar(x=param_names, y=qr_result.params[:, i],
                         marker_color='steelblue')],
            name=f'{tau:.2f}',
        )
        for i, tau in enumerate(quantiles)
    ]
    fig_slider = go.Figure(
        data=[go.Bar(x=param_names, y=qr_result.params[:, 2],
                     marker_color='steelblue')],
        frames=frames,
    )
    fig_slider.update_layout(
        title='Coefficients by Quantile',
        xaxis_title='Variable', yaxis_title='Coefficient',
        template='plotly_white', font=dict(family='Arial', size=13),
        sliders=[{
            'steps': [
                {'method': 'animate', 'label': f'τ={tau:.2f}',
                 'args': [[f'{tau:.2f}'], {'frame': {'duration': 0}, 'mode': 'immediate'}]}
                for tau in quantiles
            ],
            'currentvalue': {'prefix': 'Quantile τ = '},
            'pad': {'t': 50},
        }],
    )
    fig_slider.show()
    fig_slider.write_html('../outputs/reports/html/03_quantile_dashboard.html',
                          include_plotlyjs=True)
    print('Interactive dashboard saved.')

### Exercise 2 — Solution: SurfacePlotter coefficient heatmap

In [ ]:
# Exercise 2 solution
from panelbox.visualization.quantile.surface_plots import SurfacePlotter

sp = SurfacePlotter(figsize=(10, 6), colormap='viridis')

try:
    fig_hm = sp.coefficient_heatmap(result=qr_result, var_names=param_names)
    fig_hm.suptitle('Coefficient Heatmap: β(variable, τ)', y=1.02)
    fig_hm.tight_layout()
    fig_hm.savefig('../outputs/charts/png/03_qr_coef_heatmap.png',
                   dpi=150, bbox_inches='tight')
    print('Heatmap saved.')
    plt.show()
except Exception as e:
    print(f'SurfacePlotter error: {e} — manual heatmap.')
    # Manual heatmap using params matrix
    fig, ax = plt.subplots(figsize=(9, 4))
    im = ax.imshow(qr_result.params, aspect='auto', cmap='RdBu_r',
                   origin='upper')
    ax.set_xticks(range(len(quantiles)))
    ax.set_xticklabels([f'τ={q:.2f}' for q in quantiles])
    ax.set_yticks(range(len(param_names)))
    ax.set_yticklabels(param_names)
    ax.set_title('Coefficient Heatmap: β(variable, τ)')
    plt.colorbar(im, ax=ax, label='Coefficient value')
    fig.tight_layout()
    fig.savefig('../outputs/charts/png/03_qr_coef_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()

---
## Part 3: Spatial Visualizations

In [ ]:
from scipy.spatial.distance import cdist

df_spatial = make_spatial_data(n_obs=100, seed=42)
print(df_spatial.head())

# Row-standardized distance-band weight matrix (threshold = 2 coordinate units)
coords   = df_spatial[['x', 'y']].values
dist_mat = cdist(coords, coords)
W_raw    = (dist_mat < 2.0).astype(float)
np.fill_diagonal(W_raw, 0)
row_sums = W_raw.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
W        = W_raw / row_sums

print(f'\nW shape: {W.shape}')
print(f'Average neighbors: {(W > 0).sum(axis=1).mean():.1f}')

### 3.1 Moran Scatterplot

In [ ]:
from panelbox.visualization.spatial_plots import create_moran_scatterplot

fig_moran = create_moran_scatterplot(
    values=df_spatial['price'].values,
    W=W,
    title='Moran Scatterplot — House Prices',
    xlabel='Standardized Price',
    ylabel='Spatial Lag of Standardized Price',
    show_regression=True,
    show_quadrants=True,
)
fig_moran.tight_layout()
fig_moran.savefig('../outputs/charts/png/03_spatial_moran.png', dpi=150, bbox_inches='tight')
print('Moran scatterplot saved.')
plt.show()

# Manual Moran's I
prices   = df_spatial['price'].values
z        = (prices - prices.mean()) / prices.std()
Wz       = W @ z
n        = len(z)
morans_i = (n / W.sum()) * (z @ Wz) / (z @ z)
print(f"\nMoran's I = {morans_i:.4f}  (slope of regression line in scatterplot)")
print("Positive → positive spatial autocorrelation: similar prices cluster geographically.")

### 3.2 Spatial Weights Network

In [ ]:
# Build Plotly network: edges = neighbor pairs, nodes colored by price
edge_x, edge_y = [], []
for i in range(len(coords)):
    for j in range(i + 1, len(coords)):
        if W[i, j] > 0:
            edge_x.extend([coords[i, 0], coords[j, 0], None])
            edge_y.extend([coords[i, 1], coords[j, 1], None])

fig_net = go.Figure()
fig_net.add_trace(go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=0.6, color='#aaa'), hoverinfo='none',
    showlegend=False,
))
fig_net.add_trace(go.Scatter(
    x=coords[:, 0], y=coords[:, 1], mode='markers',
    marker=dict(
        size=9, color=df_spatial['price'].values, colorscale='RdYlGn',
        showscale=True, colorbar=dict(title='Price'),
        line=dict(width=0.5, color='white'),
    ),
    hovertemplate='Price: %{marker.color:.1f}<br>(%{x:.2f}, %{y:.2f})<extra></extra>',
    name='Houses',
))
fig_net.update_layout(
    title='Spatial Weights Network — House Prices',
    xaxis_title='X coordinate', yaxis_title='Y coordinate',
    template='plotly_white', font=dict(family='Arial', size=12),
    width=750, height=600,
)
fig_net.show()

### 3.3 Spatial Weights Heatmap

In [ ]:
from panelbox.visualization.spatial_plots import plot_spatial_weights_structure

fig_ws = plot_spatial_weights_structure(
    W=W[:30, :30],
    title='Spatial Weights Matrix W (first 30 obs)',
    figsize=(8, 8),
    cmap='Blues',
    show_colorbar=True,
)
fig_ws.tight_layout()
plt.show()

### 3.4 Choropleth Map (optional)

In [ ]:
try:
    import geopandas as gpd
    from shapely.geometry import Point

    gdf = gpd.GeoDataFrame(
        df_spatial,
        geometry=[Point(x, y) for x, y in zip(df_spatial['x'], df_spatial['y'])],
        crs='EPSG:4326',
    )
    fig_choro, ax = plt.subplots(figsize=(9, 7))
    gdf.plot(column='price', cmap='RdYlGn', legend=True, ax=ax, markersize=60)
    ax.set_title('House Prices — Spatial Distribution')
    ax.set_xlabel('X coordinate'); ax.set_ylabel('Y coordinate')
    plt.tight_layout()
    fig_choro.savefig('../outputs/charts/png/03_spatial_choropleth.png',
                      dpi=150, bbox_inches='tight')
    print('Choropleth saved.')
    plt.show()
except ImportError:
    print('geopandas not installed — choropleth skipped.  pip install geopandas')

---
## Part 4: Model Comparison Visualizations

In [ ]:
from panelbox.models.static import PooledOLS, FixedEffects, RandomEffects

df_comp       = generate_panel_data(n_individuals=40, n_periods=15, n_covariates=1, seed=42)
df_comp_reset = df_comp.reset_index()

results_ols = PooledOLS(
    formula='y ~ x1', data=df_comp_reset, entity_col='entity', time_col='time'
).fit()

results_fe = FixedEffects(
    formula='y ~ x1', data=df_comp_reset, entity_col='entity', time_col='time',
    entity_effects=True,
).fit()

results_re = RandomEffects(
    formula='y ~ x1', data=df_comp_reset, entity_col='entity', time_col='time'
).fit()

print('All 3 models estimated.')
print(f'OLS  β_x1 = {results_ols.params["x1"]:.4f}')
print(f'FE   β_x1 = {results_fe.params["x1"]:.4f}')
print(f'RE   β_x1 = {results_re.params["x1"]:.4f}')

In [ ]:
from panelbox.visualization.api import create_comparison_charts

# Correct chart type strings: 'coefficients', 'forest_plot', 'fit_comparison', 'ic_comparison'
# forest_plot requires a single model — generate separately
comparison = create_comparison_charts(
    results_list=[results_ols, results_fe, results_re],
    names=['Pooled OLS', 'Fixed Effects', 'Random Effects'],
    theme='academic',
    charts=['coefficients', 'fit_comparison', 'ic_comparison'],
)

# Forest plot with single model (FE)
forest = create_comparison_charts(
    results_list=[results_fe],
    names=['Fixed Effects'],
    theme='academic',
    charts=['forest_plot'],
)
if 'forest_plot' in forest:
    comparison['forest_plot'] = forest['forest_plot']

print(f'Charts generated: {list(comparison.keys())}')

In [ ]:
# Coefficient comparison
for key, label in [('coefficients', 'coef'), ('forest_plot', 'forest'),
                   ('fit_comparison', 'fit'), ('ic_comparison', 'ic')]:
    if key in comparison:
        print(f'\n--- {key} ---')
        # Chart objects have a .figure attribute (Plotly Figure)
        comparison[key].figure.show()
        try:
            comparison[key].figure.write_image(
                f'../outputs/charts/png/03_comparison_{label}.png', width=900, height=500
            )
            print(f'  Saved: 03_comparison_{label}.png')
        except Exception as e:
            print(f'  Save failed (pip install kaleido): {e}')
    else:
        print(f'Key "{key}" not in result. Available: {list(comparison.keys())}')

### 4.2 Interactive Dropdown Comparison

In [ ]:
model_names   = ['Pooled OLS', 'Fixed Effects', 'Random Effects']
model_results = [results_ols, results_fe, results_re]

fig_drop = go.Figure()
colors   = ['steelblue', 'coral', 'seagreen']

for i, (name, res) in enumerate(zip(model_names, model_results)):
    params = res.params
    ci     = res.conf_int(alpha=0.05)
    x_vars = list(params.index)
    y_vals = [params[p] for p in x_vars]
    err_p  = [ci.loc[p, 'upper'] - params[p] for p in x_vars]
    err_m  = [params[p] - ci.loc[p, 'lower'] for p in x_vars]

    fig_drop.add_trace(go.Bar(
        x=x_vars, y=y_vals,
        name=name, visible=(i == 1),   # FE visible by default
        error_y=dict(type='data', array=err_p, arrayminus=err_m, visible=True),
        marker_color=colors[i],
    ))

buttons = [
    dict(label=name, method='update',
         args=[{'visible': [j == i for j in range(len(model_names))]}])
    for i, name in enumerate(model_names)
]

fig_drop.update_layout(
    title='Coefficients with 95% CI — select model via dropdown',
    xaxis_title='Variable', yaxis_title='Coefficient',
    template='plotly_white', font=dict(family='Arial', size=13),
    updatemenus=[dict(buttons=buttons, showactive=True,
                      x=0.05, y=1.15, direction='down')],
)
fig_drop.show()

fig_drop.write_html('../outputs/reports/html/03_interactive_comparison.html',
                    include_plotlyjs=True)
print('Interactive comparison dashboard saved.')

### Exercise 3 — Solution: Add two-way FE model

In [ ]:
# Exercise 3 solution
# Two-way fixed effects: entity + time effects
results_twfe = FixedEffects(
    formula='y ~ x1',
    data=df_comp_reset,
    entity_col='entity',
    time_col='time',
    entity_effects=True,
    time_effects=True,     # ← add time effects
).fit()

print(f'Two-way FE  β_x1 = {results_twfe.params["x1"]:.4f}')
print(f'One-way FE  β_x1 = {results_fe.params["x1"]:.4f}')
diff = results_twfe.params['x1'] - results_fe.params['x1']
print(f'Difference = {diff:.4f}  (time effects matter if this is large)')

# Add to a 4-model dropdown chart
all_names   = ['Pooled OLS', 'FE (entity)', 'Random Effects', 'FE (two-way)']
all_results = [results_ols, results_fe, results_re, results_twfe]
all_colors  = ['steelblue', 'coral', 'seagreen', 'purple']

fig4 = go.Figure()
for i, (name, res) in enumerate(zip(all_names, all_results)):
    params = res.params
    ci     = res.conf_int(alpha=0.05)
    x_vars = list(params.index)
    y_vals = [params[p] for p in x_vars]
    err_p  = [ci.loc[p, 'upper'] - params[p] for p in x_vars]
    err_m  = [params[p] - ci.loc[p, 'lower'] for p in x_vars]
    fig4.add_trace(go.Bar(
        x=x_vars, y=y_vals, name=name,
        visible=(i == 1),
        error_y=dict(type='data', array=err_p, arrayminus=err_m, visible=True),
        marker_color=all_colors[i],
    ))

fig4.update_layout(
    title='Coefficients — 4 Models',
    xaxis_title='Variable', yaxis_title='Coefficient',
    template='plotly_white', font=dict(family='Arial', size=13),
    updatemenus=[dict(
        buttons=[
            dict(label=n, method='update',
                 args=[{'visible': [j == i for j in range(4)]}])
            for i, n in enumerate(all_names)
        ],
        showactive=True, x=0.05, y=1.20, direction='down',
    )],
)
fig4.show()

---
## Summary

| Part | Visualization | Key function / class | Critical API note |
|---|---|---|---|
| VAR | IRF grid | `plot_irf(irf_result, ...)` | `periods=` not `horizon=` |
| VAR | FEVD stacked area | `plot_fevd(fevd_result, ...)` | `periods=` not `horizon=` |
| VAR | Stability unit circle | `var_results.plot_stability()` | method on result, not standalone |
| VAR | Granger network | `var_results.plot_causality_network()` | method on result |
| Quantile | Coefficient path β(τ) | `QuantileVisualizer.coefficient_path()` | `params` shape: (n_params, n_quantiles) |
| Quantile | Fan chart | `QuantileVisualizer.fan_chart()` | `X_forecast` shape: (n_points, n_params) |
| Quantile | Interactive dashboard | `InteractivePlotter.coefficient_dashboard()` | Plotly slider fallback available |
| Quantile | Heatmap | `SurfacePlotter.coefficient_heatmap()` | class name is `SurfacePlotter` |
| Spatial | Moran scatterplot | `create_moran_scatterplot(values, W)` | returns `matplotlib.Figure` |
| Spatial | Weights heatmap | `plot_spatial_weights_structure(W)` | returns `matplotlib.Figure` |
| Spatial | Network | Plotly `Scatter` edge traces | manual construction |
| Comparison | Coefficient bars | `create_comparison_charts(charts=['coefficients'])` | NOT `'coefficient_comparison'` |
| Comparison | Forest plot | `create_comparison_charts(charts=['forest_plot'])` | same function |
| Comparison | Fit stats | `create_comparison_charts(charts=['fit_comparison'])` | NOT `'model_fit'` |
| Comparison | IC plot | `create_comparison_charts(charts=['ic_comparison'])` | same function |

**Next:** Notebook 04 — Automated Report Generation